#  UR5 Manipulator Sensor Data Cleaning – Notebook 02

**Objective:**  
This notebook performs **data merging and cleaning** for UR5 robotic manipulator controller logs.  
It integrates header metadata, multiple raw sensor CSV files, and summary deviation information into a **structured dataset** for further preprocessing in Notebook 02.

**Data Sources:**
- `ur5testresult_header.xlsx`: Defines column names, types, and meanings.
- `calculated_deviation_of_actual_position_to_nominal_position.xlsx`: Contains deviation summaries (validation reference).
- Multiple `.csv` sensor data files: Controller-level logs at 125Hz across various payloads and test conditions.

**Output:**  
- Structured dataset (columns named, concatenated, raw but organized).  
- Saved to `../data/interim/structured_sensor_data.csv` and `.parquet` for downstream use.


In [9]:
# Core Libraries
import os
import pandas as pd
import numpy as np

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [10]:
# Base directories
raw_dir = "../data/raw"
header_path = os.path.join(raw_dir, "header", "ur5testresult_header.xlsx")
summary_path = os.path.join(raw_dir, "summary", "calculated_deviation_of_actual_position_to_nominal_position.xlsx")

# Folder containing sensor CSV files
sensor_data_dir = os.path.join(raw_dir, "sensor_data")

# Output directory
output_dir = "../data/interim"
os.makedirs(output_dir, exist_ok=True)


## 📘 Header File Inspection

The header file provides metadata describing each data channel collected from the UR5 controller.  
It includes the **column name, data type, number of values per channel, and measurement notes**.


In [11]:
import pandas as pd

# Path to header file
header_path = "../data/raw/header/ur5testresult_header.xlsx"

# Load header Excel file
header_df = pd.read_excel(header_path)

# Extract column names directly
header_columns = header_df.columns.tolist()

print(f"✅ Total columns generated from header: {len(header_columns)}")
print(header_columns[:15])  # Preview first few columns


✅ Total columns generated from header: 73
['ROBOT_TIME', 'ROBOT_TARGET_JOINT_POSITIONS (J1)', 'ROBOT_TARGET_JOINT_POSITIONS (J2)', 'ROBOT_TARGET_JOINT_POSITIONS (J3)', 'ROBOT_TARGET_JOINT_POSITIONS (J4)', 'ROBOT_TARGET_JOINT_POSITIONS (J5)', 'ROBOT_TARGET_JOINT_POSITIONS (J6)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J1)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J2)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J3)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J4)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J5)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J6)', 'ROBOT_TARGET_JOINT_VELOCITIES (J1)', 'ROBOT_TARGET_JOINT_VELOCITIES (J2)']


In [12]:
import glob

# Path to raw sensor CSVs
sensor_files = glob.glob("../data/raw/sensor_data/*.csv")

# Load all CSVs with header columns applied
dataframes = []
for file in sensor_files:
    df = pd.read_csv(file, header=None, names=header_columns)
    dataframes.append(df)

# Combine all sensor data
sensor_data = pd.concat(dataframes, ignore_index=True)
print(f"✅ Combined sensor data shape: {sensor_data.shape}")
display(sensor_data.head())

✅ Combined sensor data shape: (153658, 73)


,ROBOT_TIME,ROBOT_TARGET_JOINT_POSITIONS (J1),ROBOT_TARGET_JOINT_POSITIONS (J2),ROBOT_TARGET_JOINT_POSITIONS (J3),ROBOT_TARGET_JOINT_POSITIONS (J4),ROBOT_TARGET_JOINT_POSITIONS (J5),ROBOT_TARGET_JOINT_POSITIONS (J6),ROBOT_ACTUAL_JOINT_POSITIONS (J1),ROBOT_ACTUAL_JOINT_POSITIONS (J2),ROBOT_ACTUAL_JOINT_POSITIONS (J3),ROBOT_ACTUAL_JOINT_POSITIONS (J4),ROBOT_ACTUAL_JOINT_POSITIONS (J5),ROBOT_ACTUAL_JOINT_POSITIONS (J6),ROBOT_TARGET_JOINT_VELOCITIES (J1),ROBOT_TARGET_JOINT_VELOCITIES (J2),ROBOT_TARGET_JOINT_VELOCITIES (J3),ROBOT_TARGET_JOINT_VELOCITIES (J4),ROBOT_TARGET_JOINT_VELOCITIES (J5),ROBOT_TARGET_JOINT_VELOCITIES (J6),ROBOT_ACTUAL_JOINT_VELOCITIES (J1),ROBOT_ACTUAL_JOINT_VELOCITIES (J2),ROBOT_ACTUAL_JOINT_VELOCITIES (J3),ROBOT_ACTUAL_JOINT_VELOCITIES (J4),ROBOT_ACTUAL_JOINT_VELOCITIES (J5),ROBOT_ACTUAL_JOINT_VELOCITIES (J6),ROBOT_TARGET_JOITN_CURRENT (J1),ROBOT_TARGET_JOITN_CURRENT (J2),ROBOT_TARGET_JOITN_CURRENT (J3),ROBOT_TARGET_JOITN_CURRENT (J4),ROBOT_TARGET_JOITN_CURRENT (J5),ROBOT_TARGET_JOITN_CURRENT (J6),ROBOT_ACTUAL_JOINT_CURRENT (J1),ROBOT_ACTUAL_JOINT_CURRENT (J2),ROBOT_ACTUAL_JOINT_CURRENT (J3),ROBOT_ACTUAL_JOINT_CURRENT (J4),ROBOT_ACTUAL_JOINT_CURRENT (J5),ROBOT_ACTUAL_JOINT_CURRENT (J6),ROBOT_TARGET_JOINT_ACCELERATIONS (J1),ROBOT_TARGET_JOINT_ACCELERATIONS (J2),ROBOT_TARGET_JOINT_ACCELERATIONS (J3),ROBOT_TARGET_JOINT_ACCELERATIONS (J4),ROBOT_TARGET_JOINT_ACCELERATIONS (J5),ROBOT_TARGET_JOINT_ACCELERATIONS (J6),ROBOT_TARGET_JOINT_TORQUES (J1),ROBOT_TARGET_JOINT_TORQUES (J2),ROBOT_TARGET_JOINT_TORQUES (J3),ROBOT_TARGET_JOINT_TORQUES (J4),ROBOT_TARGET_JOINT_TORQUES (J5),ROBOT_TARGET_JOINT_TORQUES (J6),ROBOT_JOINT_CONTROL_CURRENT (J1),ROBOT_JOINT_CONTROL_CURRENT (J2),ROBOT_JOINT_CONTROL_CURRENT (J3),ROBOT_JOINT_CONTROL_CURRENT (J4),ROBOT_JOINT_CONTROL_CURRENT (J5),ROBOT_JOINT_CONTROL_CURRENT (J6),ROBOT_CARTESIAN_COORD_TOOL (x),ROBOT_CARTESIAN_COORD_TOOL (y),ROBOT_CARTESIAN_COORD_TOOL (z),ROBOT_CARTESIAN_COORD_TOOL (rx),ROBOT_CARTESIAN_COORD_TOOL (ry),ROBOT_CARTESIAN_COORD_TOOL (rz),ROBOT_TCP_FORCE (x),ROBOT_TCP_FORCE (y),ROBOT_TCP_FORCE (z),ROBOT_TCP_FORCE (rx),ROBOT_TCP_FORCE (ry),ROBOT_TCP_FORCE (rz),ROBOT_JOINT_TEMP (J1),ROBOT_JOINT_TEMP (J2),ROBOT_JOINT_TEMP (J3),ROBOT_JOINT_TEMP (J4),ROBOT_JOINT_TEMP (J5),ROBOT_JOINT_TEMP (J6)
0,([13867.472],[-26.880068716264294,-79.911609,57.095392,-157.771764,-105.009613,-44.72477900700451],[-26.881428894723115,-79.910908,57.093360,-157.773152,-105.006854,-44.72477900700451],[0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,-0.0,0.0,0.0,0.0],[2.433830495651467e-17,-2.213814,-1.589348,-0.162991,0.000451,0.0],[-0.2914353609085083,-2.640853,-2.082642,-0.369060,-0.109803,0.0030500823631882668],[0.0,0.0,0.0,0.0,0.0,0.0],[2.850164985011539e-16,-25.674171,-18.441131,-1.376068,0.003848,0.0],[-0.2914353609085083,-2.64982,-2.080400,-0.36296,-0.109803,-0.0091502470895648],[-0.6376844833673514,0.277580,0.757032,-1.075050,-1.130220,0.04546741483982896],[1.7623931851118197,-6.732058,-14.279688,0.917702,0.978188,0.022847212757561457],[27.37999153137207,28.789991,28.929893,33.399982,33.393852,34.92998123168945])
1,([13867.48],[-26.880068716264294,-79.911609,57.095392,-157.771764,-105.009613,-44.72477900700451],[-26.87937983797211,-79.909542,57.091284,-157.773835,-105.008930,-44.72614504483851],[0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,-0.0,0.0,0.0,0.0],[2.433830495651467e-17,-2.213814,-1.589348,-0.162991,0.000451,0.0],[-0.2847099304199219,-2.649820,-2.066949,-0.367535,-0.115903,0.0045751235447824],[0.0,0.0,0.0,0.0,0.0,0.0],[2.850164985011539e-16,-25.674171,-18.441131,-1.376068,0.003848,0.0],[-0.2914353609085083,-2.64982,-2.078158,-0.36296,-0.109803,-0.0091502470895648],[-0.6376930015017775,0.277567,0.757041,-1.075025,-1.130212,0.04545389358637484],[0.9792112067350713,-6.221806,-13.364465,0.840332,0.869831,0.04245950647598208],[27.37999153137207,28.789991,28.929893,33.399982,33.394466,34.92998123168945])
2,([13867.488],[-26.880068716264294,-79.911609,57.095392,-157.771764,-105.009613,-44.72477900700451],[-26.88

In [13]:
sensor_data.info()
print("\nMemory usage (MB):", round(sensor_data.memory_usage().sum() / 1024 ** 2, 2))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153658 entries, 0 to 153657
Data columns (total 73 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   ROBOT_TIME                             153658 non-null  object 
 1   ROBOT_TARGET_JOINT_POSITIONS (J1)      153658 non-null  object 
 2   ROBOT_TARGET_JOINT_POSITIONS (J2)      153658 non-null  float64
 3   ROBOT_TARGET_JOINT_POSITIONS (J3)      153658 non-null  float64
 4   ROBOT_TARGET_JOINT_POSITIONS (J4)      153658 non-null  float64
 5   ROBOT_TARGET_JOINT_POSITIONS (J5)      153658 non-null  float64
 6   ROBOT_TARGET_JOINT_POSITIONS (J6)      153658 non-null  object 
 7   ROBOT_ACTUAL_JOINT_POSITIONS (J1)      153658 non-null  object 
 8   ROBOT_ACTUAL_JOINT_POSITIONS (J2)      153658 non-null  float64
 9   ROBOT_ACTUAL_JOINT_POSITIONS (J3)      153658 non-null  float64
 10  ROBOT_ACTUAL_JOINT_POSITIONS (J4)      153658 non-null  

In [14]:
# --- Define the generalized cleaning function ---
def aggressive_cleanup_to_float(df):
    """
    Identifies all object columns that contain string representations of numbers,
    aggressively cleans the strings to remove non-numeric artifacts, and converts 
    them to float64.
    """
    # 1. Select all columns currently stored as 'object'
    object_cols = df.select_dtypes(include=['object']).columns

    if len(object_cols) == 0:
        print("✅ No object columns found. Data types are already clean.")
        return df

    print(f"🧹 Applying aggressive cleanup to {len(object_cols)} object columns...")
    
    for col in object_cols:
        # 2. Start with the data as strings.
        clean_strings = df[col].astype(str)

        # 3. Aggressive Cleaning: Remove all non-numeric artifacts.
        # Regex: [^\d.\-] removes EVERYTHING that is NOT a digit, decimal point, or negative sign.
        clean_strings = (
            clean_strings
            .str.strip()
            .str.replace(r'[^\d.\-]', '', regex=True)
        )

        # 4. Convert to numeric.
        fixed_data = pd.to_numeric(clean_strings, errors='coerce')

        # 5. Overwrite the column in the DataFrame.
        df[col] = fixed_data
        
    print("✅ Cleanup complete.")
    return df

# --- Apply the function to your DataFrame ---
# We use 'sensor_data' as that is the name of the DataFrame created in the preceding code.
sensor_data = aggressive_cleanup_to_float(sensor_data)

# --- Final Validation ---
print("\n📊 Final Data Types after Full Cleanup:")
sensor_data.info(memory_usage='deep')

🧹 Applying aggressive cleanup to 25 object columns...
✅ Cleanup complete.

📊 Final Data Types after Full Cleanup:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153658 entries, 0 to 153657
Data columns (total 73 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   ROBOT_TIME                             153658 non-null  float64
 1   ROBOT_TARGET_JOINT_POSITIONS (J1)      153658 non-null  float64
 2   ROBOT_TARGET_JOINT_POSITIONS (J2)      153658 non-null  float64
 3   ROBOT_TARGET_JOINT_POSITIONS (J3)      153658 non-null  float64
 4   ROBOT_TARGET_JOINT_POSITIONS (J4)      153658 non-null  float64
 5   ROBOT_TARGET_JOINT_POSITIONS (J5)      153658 non-null  float64
 6   ROBOT_TARGET_JOINT_POSITIONS (J6)      153658 non-null  float64
 7   ROBOT_ACTUAL_JOINT_POSITIONS (J1)      153658 non-null  float64
 8   ROBOT_ACTUAL_JOINT_POSITIONS (J2)      153658 non-null  float64
 9   ROBOT_ACTUA

In [15]:
sensor_data.describe().T.head(20)


,count,mean,std,min,25%,50%,75%,max
ROBOT_TIME,153658.0,8681.076197,5974.607357,306.168000,842.066000,11874.828000,14036.678000,14569.992000
ROBOT_TARGET_JOINT_POSITIONS (J1),153658.0,-22.652577,13.589124,-50.161723,-31.880852,-24.872758,-11.241634,-2.749167
ROBOT_TARGET_JOINT_POSITIONS (J2),153658.0,-76.852773,25.580908,-128.227471,-87.965944,-78.782013,-51.054105,-39.513243
ROBOT_TARGET_JOINT_POSITIONS (J3),153658.0,81.819492,29.760853,43.397365,57.095392,65.318503,114.759010,121.363974
ROBOT_TARGET_JOINT_POSITIONS (J4),153658.0,-182.229698,19.303400,-217.771874,-200.110749,-182.113084,-164.434336,-157.766322
ROBOT_TARGET_JOINT_POSITIONS (J5),153658.0,-105.600571,3.924726,-112.186257,-109.225047,-105.009613,-101.962278,-100.242644
ROBOT_TARGET_JOINT_POSITIONS (J6),153658.0,-44.722105,0.001983,-44.728904,-44.723509,-44.722463,-44.720065,-44.718605
ROBOT_ACTUAL_JOINT_POSITIONS (J1),153658.0,-22.653296,13.590820,-50.179532,-31.881066,-24.880716,-11.238902,-2.732612
ROBOT_ACTUAL_JOINT_POSITIONS (J2),153658.0,-76.853514,25.582974,-128.248485,-87.966597,-78.784392,-51.046099,-39.507717
ROBOT_ACTUAL_JOINT_POSITIONS (J3),153658.0,81.820270,29.761970,43.392574,57.095409,65.315733,114.755538,121.372270


In [17]:
# --- Step 4: Summary Stats & Missing Values ---
print("\n⚙️ Dataset Summary:")
display(sensor_data.describe(include='all').transpose().head(73))

print("\nMissing Values:")
missing_ratio = sensor_data.isna().mean().sort_values(ascending=False)
display(missing_ratio.head(20))


⚙️ Dataset Summary:


,count,mean,std,min,25%,50%,75%,max
ROBOT_TIME,153658.0,8.681076e+03,5974.607357,306.168000,842.066000,11874.828000,14036.678000,14569.992000
ROBOT_TARGET_JOINT_POSITIONS (J1),153658.0,-2.265258e+01,13.589124,-50.161723,-31.880852,-24.872758,-11.241634,-2.749167
ROBOT_TARGET_JOINT_POSITIONS (J2),153658.0,-7.685277e+01,25.580908,-128.227471,-87.965944,-78.782013,-51.054105,-39.513243
ROBOT_TARGET_JOINT_POSITIONS (J3),153658.0,8.181949e+01,29.760853,43.397365,57.095392,65.318503,114.759010,121.363974
ROBOT_TARGET_JOINT_POSITIONS (J4),153658.0,-1.822297e+02,19.303400,-217.771874,-200.110749,-182.113084,-164.434336,-157.766322
ROBOT_TARGET_JOINT_POSITIONS (J5),153658.0,-1.056006e+02,3.924726,-112.186257,-109.225047,-105.009613,-101.962278,-100.242644
ROBOT_TARGET_JOINT_POSITIONS (J6),153658.0,-4.472210e+01,0.001983,-44.728904,-44.723509,-44.722463,-44.720065,-44.718605
ROBOT_ACTUAL_JOINT_POSITIONS (J1),153658.0,-2.265330e+01,13.590820,-50.179532,-31.881066,-24.880716,-11.238902,-2.732612
ROBOT_ACTUAL_JOINT_POSITIONS (J2),153658.0,-7.685351e+01,25.582974,-128.248485,-87.966597,-78.784392,-51.046099,-39.507717
ROBOT_ACTUAL_JOINT_POSITIONS (J3),153658.0,8.182027e+01,29.761970,43.392574,57.095409,65.315733,114.755538,121.372270



Missing Values:


ROBOT_TARGET_JOINT_VELOCITIES (J6)       0.630595
ROBOT_TARGET_JOINT_TORQUES (J6)          0.574575
ROBOT_TARGET_JOINT_ACCELERATIONS (J6)    0.537831
ROBOT_TARGET_JOINT_TORQUES (J1)          0.362103
ROBOT_TARGET_JOITN_CURRENT (J1)          0.362090
ROBOT_TARGET_JOITN_CURRENT (J6)          0.283636
ROBOT_ACTUAL_JOINT_VELOCITIES (J1)       0.113349
ROBOT_ACTUAL_JOINT_VELOCITIES (J6)       0.043239
ROBOT_TARGET_JOINT_VELOCITIES (J1)       0.003098
ROBOT_TARGET_JOINT_ACCELERATIONS (J1)    0.002356
ROBOT_CARTESIAN_COORD_TOOL (rz)          0.000169
ROBOT_TCP_FORCE (rz)                     0.000013
ROBOT_TCP_FORCE (x)                      0.000013
ROBOT_ACTUAL_JOINT_POSITIONS (J2)        0.000000
ROBOT_TARGET_JOINT_POSITIONS (J3)        0.000000
ROBOT_TARGET_JOINT_POSITIONS (J2)        0.000000
ROBOT_TARGET_JOINT_POSITIONS (J1)        0.000000
ROBOT_TIME                               0.000000
ROBOT_ACTUAL_JOINT_POSITIONS (J1)        0.000000
ROBOT_TARGET_JOINT_POSITIONS (J6)        0.000000


In [19]:
display(sensor_data.head())

,ROBOT_TIME,ROBOT_TARGET_JOINT_POSITIONS (J1),ROBOT_TARGET_JOINT_POSITIONS (J2),ROBOT_TARGET_JOINT_POSITIONS (J3),ROBOT_TARGET_JOINT_POSITIONS (J4),ROBOT_TARGET_JOINT_POSITIONS (J5),ROBOT_TARGET_JOINT_POSITIONS (J6),ROBOT_ACTUAL_JOINT_POSITIONS (J1),ROBOT_ACTUAL_JOINT_POSITIONS (J2),ROBOT_ACTUAL_JOINT_POSITIONS (J3),ROBOT_ACTUAL_JOINT_POSITIONS (J4),ROBOT_ACTUAL_JOINT_POSITIONS (J5),ROBOT_ACTUAL_JOINT_POSITIONS (J6),ROBOT_TARGET_JOINT_VELOCITIES (J1),ROBOT_TARGET_JOINT_VELOCITIES (J2),ROBOT_TARGET_JOINT_VELOCITIES (J3),ROBOT_TARGET_JOINT_VELOCITIES (J4),ROBOT_TARGET_JOINT_VELOCITIES (J5),ROBOT_TARGET_JOINT_VELOCITIES (J6),ROBOT_ACTUAL_JOINT_VELOCITIES (J1),ROBOT_ACTUAL_JOINT_VELOCITIES (J2),ROBOT_ACTUAL_JOINT_VELOCITIES (J3),ROBOT_ACTUAL_JOINT_VELOCITIES (J4),ROBOT_ACTUAL_JOINT_VELOCITIES (J5),ROBOT_ACTUAL_JOINT_VELOCITIES (J6),ROBOT_TARGET_JOITN_CURRENT (J1),ROBOT_TARGET_JOITN_CURRENT (J2),ROBOT_TARGET_JOITN_CURRENT (J3),ROBOT_TARGET_JOITN_CURRENT (J4),ROBOT_TARGET_JOITN_CURRENT (J5),ROBOT_TARGET_JOITN_CURRENT (J6),ROBOT_ACTUAL_JOINT_CURRENT (J1),ROBOT_ACTUAL_JOINT_CURRENT (J2),ROBOT_ACTUAL_JOINT_CURRENT (J3),ROBOT_ACTUAL_JOINT_CURRENT (J4),ROBOT_ACTUAL_JOINT_CURRENT (J5),ROBOT_ACTUAL_JOINT_CURRENT (J6),ROBOT_TARGET_JOINT_ACCELERATIONS (J1),ROBOT_TARGET_JOINT_ACCELERATIONS (J2),ROBOT_TARGET_JOINT_ACCELERATIONS (J3),ROBOT_TARGET_JOINT_ACCELERATIONS (J4),ROBOT_TARGET_JOINT_ACCELERATIONS (J5),ROBOT_TARGET_JOINT_ACCELERATIONS (J6),ROBOT_TARGET_JOINT_TORQUES (J1),ROBOT_TARGET_JOINT_TORQUES (J2),ROBOT_TARGET_JOINT_TORQUES (J3),ROBOT_TARGET_JOINT_TORQUES (J4),ROBOT_TARGET_JOINT_TORQUES (J5),ROBOT_TARGET_JOINT_TORQUES (J6),ROBOT_JOINT_CONTROL_CURRENT (J1),ROBOT_JOINT_CONTROL_CURRENT (J2),ROBOT_JOINT_CONTROL_CURRENT (J3),ROBOT_JOINT_CONTROL_CURRENT (J4),ROBOT_JOINT_CONTROL_CURRENT (J5),ROBOT_JOINT_CONTROL_CURRENT (J6),ROBOT_CARTESIAN_COORD_TOOL (x),ROBOT_CARTESIAN_COORD_TOOL (y),ROBOT_CARTESIAN_COORD_TOOL (z),ROBOT_CARTESIAN_COORD_TOOL (rx),ROBOT_CARTESIAN_COORD_TOOL (ry),ROBOT_CARTESIAN_COORD_TOOL (rz),ROBOT_TCP_FORCE (x),ROBOT_TCP_FORCE (y),ROBOT_TCP_FORCE (z),ROBOT_TCP_FORCE (rx),ROBOT_TCP_FORCE (ry),ROBOT_TCP_FORCE (rz),ROBOT_JOINT_TEMP (J1),ROBOT_JOINT_TEMP (J2),ROBOT_JOINT_TEMP (J3),ROBOT_JOINT_TEMP (J4),ROBOT_JOINT_TEMP (J5),ROBOT_JOINT_TEMP (J6)
0,13867.472,-26.880069,-79.911609,57.095392,-157.771764,-105.009613,-44.724779,-26.881429,-79.910908,57.093360,-157.773152,-105.006854,-44.724779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,NaN,-2.213814,-1.589348,-0.162991,0.000451,0.0,-0.291435,-2.640853,-2.082642,-0.369060,-0.109803,0.003050,0.0,0.0,0.0,0.0,0.0,0.0,NaN,-25.674171,-18.441131,-1.376068,0.003848,0.0,-0.291435,-2.64982,-2.080400,-0.36296,-0.109803,-0.00915,-0.637684,0.277580,0.757032,-1.075050,-1.130220,0.045467,1.762393,-6.732058,-14.279688,0.917702,0.978188,0.022847,27.379992,28.789991,28.929893,33.399982,33.393852,34.929981
1,13867.480,-26.880069,-79.911609,57.095392,-157.771764,-105.009613,-44.724779,-26.879380,-79.909542,57.091284,-157.773835,-105.008930,-44.726145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,NaN,-2.213814,-1.589348,-0.162991,0.000451,0.0,-0.284710,-2.649820,-2.066949,-0.367535,-0.115903,0.004575,0.0,0.0,0.0,0.0,0.0,0.0,NaN,-25.674171,-18.441131,-1.376068,0.003848,0.0,-0.291435,-2.64982,-2.078158,-0.36296,-0.109803,-0.00915,-0.637693,0.277567,0.757041,-1.075025,-1.130212,0.045454,0.979211,-6.221806,-13.364465,0.840332,0.869831,0.042460,27.379992,28.789991,28.929893,33.399982,33.394466,34.929981
2,13867.488,-26.880069,-79.911609,57.095392,-157.771764,-105.009613,-44.724779,-26.880063,-79.908859,57.091284,-157.777961,-105.008247,-44.725462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,NaN,-2.213814,-1.589348,-0.162991,0.000451,0.0,-0.295919,-2.663271,-2.066949,-0.367535,-0.123528,-0.012200,0.0,0.0,0.0,0.0,0.0,0.0,NaN,-25.674171,-18.441131,-1.376068,0.003848,0.0,-0.291435,-2.64982,-2.078158,-0.36296,-0.109803,-0.00915,-0.637693,0.277573,0.757052,-1.075000,-1.130155,0.045477,0.448985,-6.268

## 📗 Summary File Inspection

The summary file `calculated_deviation_of_actual_position_to_nominal_position.xlsx`  
contains deviation statistics between **actual and target joint positions**.  
This serves as a **validation reference** for future analysis and anomaly detection.


In [18]:
summary_df = pd.read_excel(summary_path)
print("✅ Summary file loaded successfully.\n")
display(summary_df.head())

✅ Summary file loaded successfully.



,Calculated deviation of actual position to nominal position,Unnamed: 1
0,Position,delta Distance (mm)
1,position 1,0.156205
2,position 2,0.163095
3,position 3,0.189737
4,position 4,0.17398


We perform a quick shape and column comparison to understand alignment potential  
between the summary data and structured sensor logs.


In [9]:
print("Sensor Data Columns:", len(sensor_data.columns))
print("Summary Columns:", len(summary_df.columns))


Sensor Data Columns: 1
Summary Columns: 2


At this stage, the dataset is **structurally aligned but uncleaned**.  
It will be used as input for the next notebook (`02_Data_Cleaning.ipynb`).


In [10]:
# Save both CSV and Parquet
structured_csv = os.path.join(output_dir, "structured_sensor_data.csv")
structured_parquet = os.path.join(output_dir, "structured_sensor_data.parquet")

sensor_data.to_csv(structured_csv, index=False)
sensor_data.to_parquet(structured_parquet, index=False)

print(f"✅ Structured dataset saved at:\n- {structured_csv}\n- {structured_parquet}")


✅ Structured dataset saved at:
- ../data/interim/structured_sensor_data.csv
- ../data/interim/structured_sensor_data.parquet


# ✅ Summary

- Header metadata successfully loaded and expanded (total columns: *N*).  
- All raw sensor `.csv` files concatenated into one unified DataFrame.  
- Basic data structure verified (types, memory, sample stats).  
- Summary deviation file inspected for later correlation.  
- Saved structured dataset for **Notebook 02 – Data Cleaning & Preprocessing**.
